In [3]:
import pandas, glob, penmon.eto as pm

SOURCE_DIR="data/sourcedata/"
OUTPUT_DIR="data/"

pm.CHECK_RADIATION_RANGE=False

def calculate_eto(col):
    
    try:
        day = pm.Station(col["latitude"], col["altitude"], anemometer_height=10).get_day(col["date"].strftime("%Y-%m-%d"),
                         temp_min=round(col["temp_min"], 1),
                         temp_max=round(col["temp_max"], 1),
                         wind_speed=round(col["wind_speed"],1),
                         humidity_mean=round(col["humidity_mean"] * 100, 1),
                         radiation_s=round(col["solar_radiation"], 1))
        return day.eto()
    except:
        raise Exception("Exception raised while processing line:\n" + str(col))
        
    

count = 0
dataframes=[]
all_csv_files = glob.glob(SOURCE_DIR+"*.csv")

for csv_file in all_csv_files:
    df = pandas.read_csv(csv_file, header=0,
                         names=["date", "longitude", "latitude", "altitude", "temp_max",
                               "temp_min", "precip", "wind_speed","humidity_mean", "solar_radiation", None], 
                        parse_dates=[0], index_col=None)
    
    df["eto"] = df.apply(calculate_eto, axis=1)
    dataframes.append(df)
    count += 1
    
print("Processed ", count, "files")

df=pandas.concat(dataframes)
df.to_csv(OUTPUT_DIR+"eto.csv", index=False, columns=["date", "latitude", "longitude", "altitude", "temp_min", "temp_max",
                      "wind_speed", "humidity_mean", "solar_radiation", "precip", "eto"])

Processed  53 files


In [4]:
df.head(5)

,date,longitude,latitude,altitude,temp_max,temp_min,precip,wind_speed,humidity_mean,solar_radiation,NaN,eto
0,1979-01-01,66.875,37.6236,401,16.438,6.655,0.0,3.280757,0.389999,10.437468,NaN,1.97
1,1979-01-02,66.875,37.6236,401,15.260,7.025,0.0,4.589078,0.330150,10.298419,NaN,2.45
2,1979-01-03,66.875,37.6236,401,15.336,6.856,0.0,5.436256,0.361547,10.459294,NaN,2.66
3,1979-01-04,66.875,37.6236,401,14.454,7.882,0.0,6.653929,0.374802,10.508286,NaN,2.95
4,1979-01-05,66.875,37.6236,401,16.820,7.656,0.0,5.522275,0.278040,10.887414,NaN,3.13
